In [8]:
pwd
date
git status --short
ROOT=$(git rev-parse --show-toplevel)
echo "root is: $ROOT"
OUTFOLDER=$(pwd)
echo "out folder is: $OUTFOLDER"

/auto/homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---22-25
Thu 28 Mar 22:34:46 GMT 2024
?? ./
root is: /auto/homes/sb2743/2024-borrowing/lean4
out folder is: /auto/homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---22-25


In [14]:
rm -rf $ROOT/build/noreuse || true
mkdir -p $ROOT/build/noreuse
cd $ROOT/build/noreuse

# == NO REUSE ==

echo "@@@ NOREUSE BUILD @@@"

CSVNAME=$(date +'%s---%d-%m-%Y---%H:%M:%S').noreuse
echo "CSV name is: $CSVNAME"
OUTFILE="$OUTFOLDER/$CSVNAME"
echo "output file is: $OUTFILE"

cmake ../../ \
  -DCCACHE=OFF \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=0 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFILE

make -j stage1 2>1 >> log.txt
make -j stage2 2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFILE && echo "" > $OUTFILE
/usr/bin/time -v make -j20 stage2 &> "time-noreuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage2.csv
/usr/bin/time -v make -j20 stage3 &> "time-noreuse-stage3.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage3.csv

# == REUSE ==

echo "@@@ REUSE BUILD @@@"

rm -rf $ROOT/build/reuse || true
mkdir -p $ROOT/build/reuse
cd $ROOT/build/reuse

CSVNAME="$(date +'%s---%d-%m-%Y---%H:%M:%S')"
OUTFILE="$OUTFOLDER/$CSVNAME"
echo "output file is: $OUTFILE"

cmake ../../ \
  -DCCACHE=OFF \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=1 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFILE
make -j stage1  2>1 >> log.txt
make -j stage2  2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFILE && echo "" > $OUTFILE
/usr/bin/time -v make -j stage2 &> "time-reuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage2.csv

/usr/bin/time -v make -j stage3 &> "time-noreuse-stage3.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage3.csv

echo "DONE!"
curl -d "Done benchmark: $OUTFOLDER" ntfy.sh/xISSztEV8EoOchM2

@@@ NOREUSE BUILD @@@
CSV name is: 1711665962---28-03-2024---22:46:02.noreuse.csv
output file is: /auto/homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---22-25/1711665962---28-03-2024---22:46:02.noreuse.csv
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /auto/homes/sb2743/2024-borrowing/lean4/build/noreuse
@@@ REUSE BUILD @@@
output file is: /auto/homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---22-25/1711667536---28-03-2024---23:12:16.reuse.csv

In [ ]:
# tail -f $ROOT/build/reuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/noreuse/time-noreuse-stage2.txt -n 35